In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rmukherjee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
with open("xgboost.txt", "r", encoding='utf8') as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 947427
First 300 letters of the data
-------


'100% Cotton Twill.\n5 Panel, Medium Profile.\nUnstructured Crown & Pre-Curved Visor.\nAdjustable Self-Material Strap With Hook And Loop Closure.\nProtective Plastic Travel Case.\nWorks With Most Audio Devices.\n48" Cord.\nProtective Plastic Travel Case.\nInterchangeable Earbud Covers.\nWorks With Most Audio '

-------
Last 300 letters of the data
-------


'Bulk\n5-14 days\nLarge\nIncludes TruColor or one standard color only on one location on both gloves\nBulk\nInventory shown for this item may be in multiple warehouse locations. Please allow for 5-14 business days for production or call our customer service team for ship date information.\nLarge\n5-14 days\n'

-------


In [4]:
jsonData = pd.read_json('marketing-points.json')

In [256]:
data = data.replace('includes', '')

In [5]:
jsonData.head()

,description,marketingPoints,name,number
0,Add a touch of personal expression to your wor...,[Inventory shown for this item may be in multi...,uni-ballÂ® 207 BLX Gel Pen,207BLX
1,Easily slips onto the heel of your shoe Option...,"[2 x CR2032 cell battery, Polybag, Offer Valid...",Cadence LED Shoe Clip,AF101
2,"Sleek, expandable zipper pouch that can fit yo...",[Includes TruColorâ„¢ on One Location Availabl...,Tempo Sports Fitness Belt,AF200
3,Bamboo Handles Complement Metal Implements wit...,[Includes one standard color or TruColorâ„¢ on...,5 Piece Bamboo BBQ Set,BBQ25
4,Implements made with FDA compliant Stainless E...,[],5 Piece BBQ Set (Bamboo) in Roll-Up Case,BBQ28


In [7]:
jsonData["text"] = jsonData["description"].astype(str) + " " +  jsonData["marketingPoints"].astype(str) + " " + jsonData["name"].astype(str)

In [39]:
dataV2 = '\n'.join(jsonData['text'].tolist())

In [40]:
import re

regex = re.compile('[^a-zA-Z]')


#dataV2Clean = regex.sub('\n', dataV2)
dataV2Clean = dataV2


In [41]:
print("Data type:", type(dataV2Clean))
print("Number of letters:", len(dataV2Clean))
print("First 300 letters of the data")
print("-------")
display(dataV2Clean[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(dataV2Clean[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 1569780
First 300 letters of the data
-------


'Add a touch of personal expression to your work with the rich, black-infused colors of the 207 BLX Formulated for consistently smooth, even ink flow, this series ensures that every word is brilliantly communicated uni Super Inkâ„¢ locks in ideas, protecting them from water, fading, & fraud Point siz'

-------
Last 300 letters of the data
-------


"any Bottles Of Water You Drink.', 'Twist Lid Every Time Bottle Is Refilled To Update The Water Symbol On Lid.'] 22 Oz. Stainless Steel Hydro Bottle\nPlunger Action. Rubberized Aluminum Pen. Unique Grip Design. Long Barrel With Shorter Pocket Clip. ['Long Barrel With Shorter Pocket Clip.'] Brinnix Pen"

-------


In [42]:
def split_to_sentences(data):
 
    sentences = data.split('\n')
   
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    
    return sentences 

In [72]:
def tokenize_sentences(sentences):
  
    tokenized_sentences = []
    
    for sentence in sentences:
        
        sentence = sentence.lower()
        sentence = sentence.replace('includes', '')
        sentence = sentence.replace('plunger', '')
        sentence = sentence.replace('laser', '')
        sentence = sentence.replace('engraving', '')
        
        
        tokenized = nltk.word_tokenize(sentence)
        
        words=[word.lower() for word in tokenized if word.isalpha() and len(word) > 2]
        
        tokenized_sentences.append(words)
    
    
    return tokenized_sentences

In [73]:
def get_tokenized_data(data):
   
    sentences = split_to_sentences(data)
    
    tokenized_sentences = tokenize_sentences(sentences)
    
    
    return tokenized_sentences

In [74]:
type(dataV2Clean)

str

In [75]:
tokenized_data = get_tokenized_data(dataV2Clean)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [76]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])
      
print("First test sample")
print(test_data[0])

5147 data are split into 4117 train and 1030 test set
First training sample:
['action', 'sleek', 'write', 'low', 'viscosity', 'ink', 'smooth', 'rubberized', 'finish', 'maverick', 'sleek', 'write', 'pen']
First test sample
['made', 'pvc', 'nfc', 'embedded', 'and', 'easily', 'share', 'fully', 'customized', 'profiles', 'containing', 'links', 'photos', 'contact', 'information', 'and', 'more', 'use', 'linq', 'app', 'share', 'profile', 'linq', 'app', 'available', 'ios', 'android', 'full', 'color', 'linq', 'digital', 'business', 'card']


In [77]:
def count_words(tokenized_sentences):
    word_counts = {}
    for sentence in tokenized_sentences: 
        for token in sentence: 
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

In [78]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items(): 
        if cnt >= count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [79]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence: 
            if token in vocabulary: 
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

In [80]:

def preprocess_data(train_data, test_data, count_threshold):      
    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)    
    train_data_replaced = replace_oov_words_by_unk(train_data,vocabulary)    
    test_data_replaced = replace_oov_words_by_unk(test_data,vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

In [81]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, 
                                                                        test_data, 
                                                                        minimum_freq)

In [82]:

def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}
    for sentence in data: 
        sentence = [start_token] * n+ sentence + [end_token]
        sentence = tuple(sentence)
        m = len(sentence) if n==1 else len(sentence)-1
        for i in range(m): 
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys(): 
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    return n_grams

In [83]:

def estimate_probability(word, previous_n_gram, 
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts  else 0
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    return probability

In [84]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    return probabilities

In [85]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [86]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [87]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
   
    n = len(list(n_gram_counts.keys())[0]) 
    
    sentence = ["<s>"] * n + sentence + ["<e>"]
    
    sentence = tuple(sentence)
    
    N = len(sentence)
    
   
    product_pi = 1.0
    
    
    for t in range(n, N): # complete this line

        n_gram = sentence[t-n:t]
        
        word = sentence[t]
       
        probability = estimate_probability(word,n_gram, n_gram_counts, n_plus1_gram_counts, len(unique_words), k=1)
        
        product_pi *= 1 / probability

    perplexity = product_pi**(1/float(N))
    
    return perplexity

In [88]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    n = len(list(n_gram_counts.keys())[0]) 
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestion = None
    max_prob = 0
    for word, prob in probabilities.items(): # complete this line
        if start_with != None: 
            if not word.startswith(start_with): 
                continue  
        if prob > max_prob: 
            suggestion = word
            max_prob = prob
    return suggestion, max_prob

In [89]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [90]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['action', 'sleek', 'write', 'low', 'viscosity', 'ink', 'smooth', 'rubberized', 'finish', '<unk>', 'sleek', 'write', 'pen']

First preprocessed test sample:
['made', 'pvc', 'nfc', 'embedded', 'and', 'easily', 'share', 'fully', 'customized', 'profiles', 'containing', 'links', 'photos', 'contact', 'information', 'and', 'more', 'use', 'linq', 'app', 'share', 'profile', 'linq', 'app', 'available', 'ios', 'android', 'full', 'color', 'linq', 'digital', 'business', 'card']

First 10 vocabulary:
['action', 'sleek', 'write', 'low', 'viscosity', 'ink', 'smooth', 'rubberized', 'finish', 'pen']

Size of vocabulary: 4480


In [91]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [92]:
previous_tokens = ["sun"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['sun'], the suggestions are:


[('spf', 0.003099402258135931),
 ('action', 0.00022311468094600624),
 ('action', 0.00022311468094600624),
 ('action', 0.00022311468094600624)]

In [93]:
previous_tokens = ["resis"]
#tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="s")
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['resis'], the suggestions are:


[('action', 0.00022311468094600624),
 ('action', 0.00022311468094600624),
 ('action', 0.00022311468094600624),
 ('action', 0.00022311468094600624)]

In [94]:
minimum_freq = 3
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, 
                                                                        test_data, 
                                                                        minimum_freq)

In [95]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [104]:
previous_tokens = [""]
#tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="s")
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['candy'], the suggestions are:


[('made', 0.004745951982132886),
 ('<e>', 0.000860832137733142),
 ('<e>', 0.000860832137733142),
 ('<e>', 0.000860832137733142)]

In [97]:
previous_tokens = ["sun", "bum", "product", "hit"]
#tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="s")
tmp_suggest9 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

In [98]:
display(tmp_suggest9)

[('promotional', 0.021739130434782608),
 ('action', 0.00028694404591104734),
 ('action', 0.00028694404591104734),
 ('action', 0.00028694404591104734)]

In [101]:
previous_tokens = ["stress"]
tmp_suggest9 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
#tmp_suggest9 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
display(tmp_suggest9)

[('reliever', 0.00846262341325811),
 ('action', 0.00028694404591104734),
 ('action', 0.00028694404591104734),
 ('action', 0.00028694404591104734)]